# Spotting deputies

## Preliminaries

In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re

# from collections import defaultdict
import pandas as pd
# import numpy as np
# #import xmltodict
# import networkx as nx
# from sqlalchemy import text


repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

import sys
!{sys.executable} -m pip install -U fuzzy_search

Requirement already up-to-date: fuzzy_search in /Users/rikhoekstra/.local/share/virtualenvs/republic-project-Av06XlOZ/lib/python3.8/site-packages (1.4.2)


In [2]:
%reload_ext numpy


In [3]:
abbr_delegates = pd.read_parquet("/Users/rikhoekstra/develop/republic_clean/republic/data/csvs/abbreviated_delegates.parquet") 

In [4]:
# let's repress the warnings, esp. from pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [5]:
import run_attendancelist

In [ ]:
from republic.data.delegate_database import abbreviated_delegates, found_delegates


In [ ]:
def overl(x, y):
    try:
        if x.overlaps(y):
            return True
        else:
            return False
    except:
        print(x, type(x))
        raise
abbreviated_delegates.loc[abbreviated_delegates.p_interval.apply(lambda x: overl(x, pd.Interval(1623, 1677)))]

In [6]:
runner = run_attendancelist.RunAll(year=1730, outdir='/Users/rikhoekstra/republic_out')


logging to /Users/rikhoekstra/republic_out/attendancelist.log


In [7]:
runner.year

1730

In [ ]:
def get_president(ob, txt):
    ofset = 0  # in case we find no president marker
    end = 0
    prez = ''
    prae = ''
    matches = ps.find_matches(text=txt)
    presidents = [m for m in matches if m.label == 'president']
    president = best_match(presidents)
    print (president)
    if president:
        ofset = getattr(president, "offset") or 0
        prez = getattr(president, "string") or ''
    presentibi = [m for m in matches if m.label == 'presentibus']
    if presentibi:
        presentibus = best_match(presentibi)
        prae = presentibus.string or ''
        end = getattr(presentibus, "offset") or len(txt)
    spns = {}
    if ofset != 0:
        preamble_span = (0, ofset - 1)
        preamble = txt[preamble_span[0]:preamble_span[1]]
        spns['preamble'] = (preamble, preamble_span)
    pre_span = (ofset, ofset + len(prez))
    spns['pre'] = (prez, pre_span)
    presentibus_span = (end, end + len(prae))
    spns['presentibus'] = (prae, presentibus_span)
    searchpat = pat % (re.escape(prez), re.escape(prae))
    print(searchpat)
    r = re.search(searchpat, txt)
    print(r)
    if r and r.group(1):
        heer = r.group(1).strip()
        heer = re.sub('[^\s\w]*', '', heer)
        #logging.info(heer)
        s1 = txt.find(heer)
        s2 = s1 + len(heer)
        span = (s1, s2)  # get_span_from_regex(r)
        spns['president'] = (heer, span)
        for kw in spns.keys():
            s = spns[kw][0]
            spn = spns[kw][1]
            ob.set_span(span=spn, pattern=s, clas=kw)
            setattr(ob, 'found', {'president': heer})
            # if prae != '':
            #     prespan = get_span_from_regex(prae)
            #     mt.set_span(prespan, "presentibus")
        return heer

In [ ]:
import random
from republic.analyser.attendance_lists.pattern_finders import president_searcher, get_president
#president_searcher(runner.searchobs)
ps = make_president_searcher()
ks = random.choices(list(runner.searchobs.keys()), k=20)
pres = []
prs = []
for T in ks:
    ob = runner.searchobs[T].matched_text
    txt = ob.item
    res = ps.find_matches(text=txt)

    presidents = [m for m in res if m.label=='president']
    pibus = [m for m in res if m.label=='presentibus']
    if presidents:
        president = best_match(presidents)
        pres.append(president)
        print('president: ', president.string)
    else:
        print('no president', T)
    if pibus:
        prbus = best_match(pibus)
        prs.append(prbus)
        print('pibus: ', prbus.string)
    else:
         print('no pibus', T)
    try:
        print('resultaat: ', gp(president, prbus)[1])
        print('getprres:', get_president(ob, r"%s(.*)%s", ps, txt))
    except (UnboundLocalError, IndexError):
        pass
    


In [ ]:
presidents

In [ ]:
import re
def gp(president, praesentibus):
    ofset = getattr(presidents[0], "offset") or 0
    prez = getattr(presidents[0], "string") or ''
    if pibus:
        presentibus = best_match(pibus)
        prae = presentibus.string or ''
        end = getattr(presentibus, "offset") or len(txt)
    searchpat = pat % (re.escape(prez), re.escape(prae))
    r = re.search(searchpat, txt)
    heer = r.group(1).strip()
    rheer = re.sub('[^\s\w]*', '', heer)
    rex
    return(heer, rheer)
#print(ofset, prez, prae, end)

In [7]:
%time runner.initial_find()

1. finding presidents
293 found
2.find provincial extraordinaris gedeputeerden
CPU times: user 37 s, sys: 1.2 s, total: 38.2 s
Wall time: 40.1 s


In [225]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel
# from ...data.delegate_database import abbreviated_delegates, found_delegates, ekwz
# from .parse_delegates import FndMatch, fndmatch, match_previous
# from .identify import iterative_search


fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": True,
    "ngram_size": 3,
    "skip_size": 1,
}
def make_province_searcher(config):
    """TODO is this faster if we first find the provinces and then find the rest???"""
    pr_searcher = FuzzyPhraseSearcher(config)
    basephrase = [{'phrase':"extraordinaris Gedeputeerden uyt de provincie van",
                  'label':'extraordinaris',
                  'variants':[]}]
    prefix = [{'phrase':'met een', 'label':'prefix','variants':['met twee', 'met drie', 'een ', 'twee ', 'drie']}]
    provinces = [{'phrase': "Gelderlandt",'label':'province',
                 'variants':["Hollandt ende West-Vrieslandt",
                 "Utrecht",
                 "Frieslandt",
                 "Overijssel",
                 "Groningen",
                 "Zeelandt"]}]
    phrases = basephrase + prefix + provinces
    pmodel = PhraseModel(model=phrases, config=fuzzysearch_config)
    pr_searcher.index_phrase_model(phrase_model=pmodel)
    return pr_searcher

In [226]:
presentielijsten = runner.searchobs
pr_searcher = make_province_searcher(fuzzysearch_config)

In [227]:
def match_prov(presentielijsten, config):
    for T in presentielijsten.keys():
        itm = presentielijsten[T]
        txt = itm.text
        print(txt)
        provs = pr_searcher.find_matches(text=txt, include_variants=True)
        print(T, provs)
            #mt.set_span(span, "province")
            

In [212]:
def match_prov(presentielijsten, config):
    for T in presentielijsten.keys():
        itm = presentielijsten[T]
        txt = itm.text
        print(txt)
        provs = pr_searcher.find_matches(text=txt, include_variants=True)
        print(T, provs)
        bstr = [s for s in provs if s.label=='extraordinaris']
        for item in bstr:
            o = item.offset
            l = o + len(item.string)
            pre_searchtext = [p.offset for p in provs if p.label=='prefix' and p.offset in range(o-10,o)]
            if len(pre_searchtext)>0:
                o = pre_searchtext[0]
            post_searchtext = [p.offset + len(p.string) for p in provs if p.label=='province' and p.offset in range(l,l+30)]
            if len(post_searchtext) > 0:
                l = post_searchtext[0]
            foundtext = txt[o:l] 
            print('found: ', foundtext)
            #mt.set_span(span, "province")
            

<pre>
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
CPU times: user 11min 42s, sys: 7.39 s, total: 11min 49s
Wall time: 11min 55s 


fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.9,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
CPU times: user 9min 31s, sys: 6.4 s, total: 9min 37s
Wall time: 9min 42s

met alleen 'extraordinaris gedeputeerde'
CPU times: user 23.4 s, sys: 371 ms, total: 23.7 s
Wall time: 23.9 s

met 'extraordinaris gedeputeerde uyt de provincie van'
CPU times: user 1min 38s, sys: 1.07 s, total: 1min 39s
Wall time: 1min 39s

CPU times: user 31.7 s, sys: 1.19 s, total: 32.9 s
Wall time: 33.8 s

en met tolerantere levenshtein distance (0.6) en ngram size 3
CPU times: user 1min 33s, sys: 843 ms, total: 1min 34s
Wall time: 1min 34s

met varianten
CPU times: user 9min 10s, sys: 4.79 s, total: 9min 15s
Wall time: 9min 17s


ngram size 3 en skipsize 1:
lange varianten ('met een extraordinaris Gedelegeerde uyt de provincie van Gelderlandt' etc)
CPU times: user 1min 55s, sys: 1.47 s, total: 1min 57s
Wall time: 1min 58s

searchstring opgedeeld in prefix ('(met) een/twee/drie'), 
basestring ('extraordinaris Gedelegeerde uyt de provincie') en provincies

CPU times: user 13.2 s, sys: 800 ms, total: 14 s
Wall time: 14.3 s

eindresult met templating obv deelstrings
CPU times: user 14 s, sys: 369 ms, total: 14.4 s
Wall time: 14.6 s

</pre>

In [ ]:
%time match_prov(presentielijsten, fuzzysearch_config)

In [ ]:
province_searcher(presentielijsten)

In [ ]:
from republic.analyser.attendance_lists.pattern_finders import province_searcher
%time province_searcher(searchobs)

In [ ]:
from republic.analyser.attendance_lists.pattern_finders import province_searcher
%time province_searcher(searchobs)

In [ ]:
from republic.analyser.attendance_lists.pattern_finders import make_province_searcher, base_config
prs = make_province_searcher(config=base_config)
for T in ks:
    ob = runner.searchobs[T].matched_text
    txt = ob.item
    print(prs.find_matches(txt, include_variants=True))

In [52]:
searchobs = runner.searchobs
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
#     if url:
#         rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


In [9]:
deputies = runner.find_unmarked_text()

3. finding unmarked text


In [10]:
connected_presidents = run_attendancelist.FuzzyKeywordGrouper(runner.presidents).vars2graph()
d = run_attendancelist.find_delegates(input=connected_presidents,
               matchfnd=runner.matchfnd,
               df=runner.abbreviated_delegates,
               previously_matched=runner.found_delegates,
               year=runner.year)

In [107]:
new_found_delegates = run_attendancelist.find_delegates(input=deputies,
                                             matchfnd=runner.matchfnd,
               df=runner.abbreviated_delegates,
               previously_matched=runner.found_delegates,
               year=runner.year)

Phrase(, None)
Phrase(, None)


In [108]:
new_found_delegates

{'matched': defaultdict(list,
             {17397: {'id': 17397,
               'm_kw': 'Quint',
               'score': 0.9,
               'name': 'Quintus',
               'variants': [('Quint', 'Quint', 1.0),
                ('Suint', 'Quint', 0.8),
                ('Ouint', 'Quint', 0.8),
                ('Quiut', 'Quint', 0.8),
                ('Jouint', 'Quint', 0.6666666666666667),
                ('&uint', 'Quint', 0.8),
                ('Vuint', 'Quint', 0.8)]},
              17821: {'id': 17821,
               'm_kw': 'Amerongen',
               'score': 0.9,
               'name': 'Borre van Amerongen',
               'variants': [('Amerongen', 'Amerongen', 1.0),
                ("a'merongen", 'a', 1.0)]},
              13477: {'id': 13477,
               'm_kw': 'Wassenaer',
               'score': 1.0,
               'name': 'van Wassenaer van Obdam',
               'variants': [('HWiassenaer', 'Wassenaer', 0.8181818181818181),
                ('Weasscnaer', 'Wassenaer', 

In [116]:
from operator import getitem
for item in new_found_delegates['unmatched']:
    if len(item) >= 1:
        print(item[0])
    else:
        try:
            print(max(item, key=lambda x: item[2]))
        except IndexError:
            print('XXX', item)

('Haayen', 'Haaften', 0.7142857142857143)
('Hssenius', 'Essenius', 0.875)
('Berchuss', 'Berchuis', 0.875)
('Beur', 'Beer', 0.75)
('QueisenDE', 'Queisen', 0.7777777777777778)
('Burgb', 'Burgh', 0.8)
('lYynbergen', 'Lengbergen', 0.6)
('Citiers', 'Citters', 0.8571428571428572)
('Clant', 'Clant', 1.0)
eer1730
('Eckvan', '\\(van', 0.5)
Februarii1730
('Griffie', 'Griffioen', 0.7777777777777778)
('Hungrosgne', 'Hurgronje', 0.6)
('Helsdingen', 'Helsdingen', 1.0)
('Helsingb', 'Helsdingen', 0.7)
('Verelst', 'Verelst', 1.0)
('Hursk', 'Hurck', 0.8)
('Torck', 'Torck', 1.0)
('Wilbem', 'Wilhem', 0.8333333333333334)
('Jongh', 'Jongh', 1.0)
('Jongemaats', 'Jongemaats', 1.0)
('Kempenaar', 'Kempenaar', 1.0)
('Lintelv', 'Lintelo', 0.8571428571428572)
('Londen', 'Linden', 0.8333333333333334)
('landt', 'Landt', 0.8)
('Lune', 'Lane', 0.75)
('Ney', 'Neyn', 0.75)
('Merck', 'Mercks', 0.8333333333333334)
('Nergena', 'Nergena', 1.0)
PURIAERSIDE
('Rambhorst', 'Ramhorst', 0.8888888888888888)
('Requeste', 'Renesse',

In [119]:
runner.matchfnd.searcher.find_matches(text='Essenius')


[PhraseMatch(phrase: "Essenius", variant: "Essenius", string: "Essenius", offset: 0, ignorecase: False, levenshtein_similarity: 1.0)]

In [125]:
run_attendancelist.iterative_search(name='Essenius', year=1730, df=df)

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval.left,p_interval.right,h_life.left,h_life.right,p_interval,h_life,score
6191,17858,Essenius,None,1714-12-17,NaN,Gedeputeerde Staten van het Kwartier Nijmegen ...,"ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1710-05-20', '1710-05-21', '1710...",True,True,1710,1735,1666,1714,"(1710, 1735]","(1666, 1714]",1.0


In [127]:
run_attendancelist.identify(name='Essenius', year=1730, df=df, window=1)

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval.left,p_interval.right,h_life.left,h_life.right,p_interval,h_life
6191,17858,Essenius,None,1714-12-17,NaN,Gedeputeerde Staten van het Kwartier Nijmegen ...,"ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1710-05-20', '1710-05-21', '1710...",True,True,1710,1735,1666,1714,"(1710, 1735]","(1666, 1714]"


In [121]:
df.loc[df.name == 'Essenius']

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval.left,p_interval.right,h_life.left,h_life.right,p_interval,h_life
94,432,Essenius,1755-01-01,1811-12-31,NaN,Staten-Generaal der Verenigde Nederlandse Prov...,gedeputeerde,"PeriodIndex(['1795-04-27', '1795-04-28', '1795...",True,True,1795,1796,1755,1811,"(1795, 1796]","(1755, 1811]"
6191,17858,Essenius,None,1714-12-17,NaN,Gedeputeerde Staten van het Kwartier Nijmegen ...,"ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1710-05-20', '1710-05-21', '1710...",True,True,1710,1735,1666,1714,"(1710, 1735]","(1666, 1714]"
6873,18621,Essenius,1728-01-01,1784-05-11,NaN,Hof van Gelre en Zutphen (1588-1795),raadsheer,"PeriodIndex(['1777-07-08', '1777-07-09', '1777...",False,False,1777,1781,1728,1784,"(1777, 1781]","(1728, 1784]"
6890,18639,Essenius,None,1766-10-31,NaN,Hof van Gelre en Zutphen (1588-1795),extraordinaris raadsheer,"PeriodIndex(['1726-01-12', '1726-01-13', '1726...",False,False,1726,1766,1692,1766,"(1726, 1766]","(1692, 1766]"


In [50]:
slsr = run_attendancelist.score_levenshtein_similarity_ratio

In [ ]:
from collections import defaultdict, Counter
previously_matched = run_attendancelist.found_delegates
iterative_search = run_attendancelist.iterative_search
df = run_attendancelist.abbreviated_delegates
year = runner.year
input = deputies
matched_deputies = defaultdict(list)
unmatched_deputies = []
for herengroup in input:
        # we add the whole group to recognized if one name has a result
        recognized_group = []
        keyword_counter = Counter()
        in_matched = False
        for heer in herengroup: # we try to fuzzymatch the whole group and give the string a score
            rslt = runner.matchfnd.match_candidates(heer=heer)
            if rslt:
                match_kw = getattr(rslt,'match_keyword')
                #print(match_kw)
                if match_kw is not '':
                    in_matched = True
                    match_distance = getattr(rslt,'levenshtein_distance')
                    recognized_group.append((heer, match_kw, match_distance))
                    keyword_counter.update([match_kw])
        if in_matched:
            kw = keyword_counter.most_common(10)[0][0]
            rec = previously_matched.loc[previously_matched.variants.apply(lambda x: matchwithlist(kw, x))]
            if len(rec) > 0:
                rec = iterative_search(name=kw, year=year, df=df)
            if len(rec) > 0:
                drec = rec.to_dict(orient="records")[0]
                m_id = drec['id']
    #             if m_id == 'matched':
    #                 print(rec, kw)
                name = drec['name']
                score = drec.get('score') or 0.0
                matched_deputies[m_id] = {'id': m_id,
                                          'm_kw':kw,
                                          'score':score,
                                          'name':name,
                                          'variants':recognized_group}
            else:
                unmatched_deputies.append(recognized_group)
        else:
            unmatched_deputies.append(herengroup)

In [86]:
def matchwithlist(x, input):
    res = [i for i in input if slsr(x, i) > 0.6]
    if len(res) > 0:
        return True
    else:
        return False
matchwithlist( 'Taats van Amerongen', xl)

True

In [88]:
previously_matched.loc[previously_matched.variants.apply(lambda x: matchwithlist( 'Taats van Amerongen', x))]

,ref_id,geboortejaar,sterfjaar,colleges,functions,period,active_stgen,was_gedeputeerde,variants,name,found_in,id,sg,period_active.left,period_active.right,hypothetical_life.left,hypothetical_life.right,p_interval.left,p_interval.right
2,17821,NaN,1640-01-21,"Ridderschap van Utrecht, Staten-Generaal der V...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1606-01-01', '1606-01-02', '1606...",True,True,"[Taats van Amerongen., Taats van Amerongen. P,...",Borre van Amerongen,1728,17821,True,1606,1640,1562,1640,1606,1640


In [101]:
matched_deputies

defaultdict(list,
            {17397: {'id': 17397,
              'm_kw': 'Quint',
              'score': 0.9,
              'name': 'Quintus',
              'variants': [('Quint', 'Quint', 1.0),
               ('Suint', 'Quint', 0.8),
               ('Ouint', 'Quint', 0.8),
               ('Quiut', 'Quint', 0.8),
               ('Jouint', 'Quint', 0.6666666666666667),
               ('&uint', 'Quint', 0.8),
               ('Vuint', 'Quint', 0.8)]},
             17821: {'id': 17821,
              'm_kw': 'Amerongen',
              'score': 0.9,
              'name': 'Borre van Amerongen',
              'variants': [('Amerongen', 'Amerongen', 1.0),
               ("a'merongen", 'a', 1.0)]},
             13477: {'id': 13477,
              'm_kw': 'Wassenaer',
              'score': 1.0,
              'name': 'van Wassenaer van Obdam',
              'variants': [('HWiassenaer', 'Wassenaer', 0.8181818181818181),
               ('Weasscnaer', 'Wassenaer', 0.8),
               ('Yassenaer', 

In [102]:
unmatched_deputies

[[('Haayen', 'Haaften', 0.7142857142857143),
  ('HWiaayen', 'Haaften', 0.5),
  ('Yaayen', 'Oyen', 0.5),
  ('Waauyen', 'Waeyen', 0.7142857142857143),
  ("H'aayen", 'Heusden', 1.0),
  ('Waayen', 'Waeyen', 0.8333333333333334),
  ("HW'aayen", 'Haaften', 0.5),
  ('Weaayen', 'Waeyen', 0.7142857142857143),
  ('Woaayen', 'Waeyen', 0.7142857142857143),
  ('aayen', 'Oyen', 0.6),
  ('Wiaayen', 'Waeyen', 0.7142857142857143),
  ('Haaayen', 'Haaften', 0.7142857142857143),
  ('YWaayen', 'Waeyen', 0.7142857142857143),
  ("B'aayen", 'Buysero', 1.0),
  ("Y'aayen", 'Ysbrands', 1.0),
  ('Wauayen', 'Waeyen', 0.7142857142857143),
  ("W'aayen", 'Weesp', 1.0)],
 [('Hssenius', 'Essenius', 0.875),
  ('B/senius', 'Buysero', 1.0),
  ('Esjenius', 'Essenius', 0.875),
  ('Fsenius', 'Essenius', 0.75),
  ('Fssenius', 'Essenius', 0.875),
  ('Essentus', 'Essenius', 0.875),
  ('Eseniut', 'Essenius', 0.75),
  ('FHisenius', 'Hixenius', 0.7777777777777778),
  ('Bssenius', 'Essenius', 0.875),
  ('Eyenius', 'Essenius', 0.75),

In [ ]:
class FndMatch(object):
    def __init__(self,
                 year=0,
                 searcher=FuzzyPhraseSearcher,
                 junksearcher=FuzzyPhraseSearcher,
                 # patterns=list,
                 register=dict,
                 rev_graph=dict,
                 df=pd.DataFrame):
        self.year = year
        self.searcher = searcher
        self.junksearcher = junksearcher
        self.register = register
        # self.patterns = patterns
        self.rev_graph = rev_graph
        self.df = df

    def match_candidates(self, heer=str):
        found_heer = Heer()
        try:
            candidates = self.searcher.find_matches(text=heer, use_word_boundaries=False, include_variants=True)
            print(candidates)
            if candidates:
                # sort the best candidate
                candidate = max(candidates, key=lambda x: x.levenshtein_similarity)
                levenshtein_similarity = candidate.levenshtein_similarity
                searchterm = candidate.phrase.exact_string
                n_candidates = self.rev_graph.get(searchterm) or []
                found_heer.levenshtein_distance = levenshtein_similarity
                found_heer.searchterm = searchterm
                found_heer.match_keyword = searchterm
                found_heer.score = 1.0
                if len(n_candidates) == 0:
                    found_heer.fill('')
                elif len(n_candidates) == 1:
                    found_heer.proposed_delegate = n_candidates[0]
                    candidate = iterative_search(name=found_heer.proposed_delegate, year=self.year, df=self.df)
                    found_heer.fill(candidate)
                else:
                    candidate = self.composite_name(candidates=n_candidates, heer='')
                    found_heer.fill(candidate)
                found_heer.probably_junk = self.is_heer_junk(heer)
        except ValueError:
            logging.info(msg=f'{heer} ValueError')
            pass

        return found_heer

    def composite_name(self, candidates=[], heer='', searchterm=str):
        dayinterval = pd.Interval(self.year, self.year, closed="both")
        candidate = dedup_candidates(proposed_candidates=candidates,
                                     register=self.register,
                                     searcher=herensearcher,
                                     searchterm=searchterm,
                                     dayinterval=dayinterval,
                                     df=self.df)
        if len(candidate) == 0:
            src = iterative_search(name=heer, year=self.year, df=self.df)
            #             if len(src) > 1:
            #                 src = src.loc[src.p_interval.apply(lambda x: x.overlaps(dayinterval))]
            #                 if len(src) == 0:
            #                     src = src.loc[src]
            #            src['levenshtein_distance'] = src.name.apply(lambda x: score_levenshtein_distance_ratio(term1=self.levenshtein_distance, term2=x))
            src.sort_values(['score', ])
            # self.heer.levenshtein_distance = self.levenshtein_distance
            candidate = src.iloc[src.index == src.first_valid_index()]  # brrrr

        return candidate

    def is_heer_junk(self, heer):
        probably_junk = False
        try:
            probably_junk_result = self.junksearcher.find_matches(text=heer, include_variants=True)
            if len(probably_junk_result) > 0:
                probably_junk = True
            return probably_junk
        except ValueError:
            logging.info(f"fuzzysearcher value error raised #{heer}#, {self.junksearcher.phrase_model.json}")
            return True


        return found_heer

In [70]:
from collections import defaultdict, Counter
input = connected_presidents
matched_deputies = defaultdict(list)
unmatched_deputies = []
for herengroup in input:
        # we add the whole group to recognized if one name has a result
        recognized_group = []
        keyword_counter = Counter()
        in_matched = False
        for heer in herengroup: # we try to fuzzymatch the whole group and give the string a score
            print(heer)
            rslt = runner.matchfnd.searcher.find_matches(text=heer, include_variants=True)
            print(rslt)
#             if rslt:
#                 in_matched = True
#                 match_kw = getattr(rslt,'match_keyword')
#                 print(match_kw)
#                 match_distance = getattr(rslt,'levenshtein_distance')
#                 recognized_group.append((heer, match_kw, match_distance))
#                 keyword_counter.update([match_kw])

Becker
[PhraseMatch(phrase: "Becker", variant: "Becker",string: "Becker", offset: 0)]
Bout
[PhraseMatch(phrase: "Bout", variant: "Bout",string: "Bout", offset: 0)]
Zaais van Amerongen
[PhraseMatch(phrase: "Zas", variant: "Zas",string: "Zaais", offset: 0), PhraseMatch(phrase: "(van", variant: "(van",string: "van", offset: 6), PhraseMatch(phrase: "Amerongen", variant: "Amerongen",string: "Amerongen", offset: 10)]
Zaatrs van Amerongen
[PhraseMatch(phrase: "Baars", variant: "Baars",string: "Zaatrs", offset: 0), PhraseMatch(phrase: "(van", variant: "(van",string: "van", offset: 7), PhraseMatch(phrase: "Amerongen", variant: "Amerongen",string: "Amerongen", offset: 11)]
Zaars van HAmerongen
[PhraseMatch(phrase: "Baars", variant: "Baars",string: "Zaars", offset: 0), PhraseMatch(phrase: "(van", variant: "(van",string: "van", offset: 6), PhraseMatch(phrase: "Amerongen", variant: "Amerongen",string: "HAmerongen", offset: 10)]
Zaats van Amerongen
[PhraseMatch(phrase: "Zas", variant: "Zas",string: 

In [71]:
x = runner.matchfnd.match_candidates(heer="Isselmuden")
x.levenshtein_distance

1.0

In [ ]:
pm = 

In [ ]:
phrasemodel = PhraseModel()

In [ ]:
pm = runner.matchfnd.searcher.phrase_model

In [75]:
r = runner.matchfnd.searcher.find_matches(text='Isselmuden', include_variants=True)
r[0].levenshtein_similarity

1.0

In [56]:
fragments = ob.get_fragments()
# fragment += "[{}]".format(i.idnr) # for now
outfragments = []
for fragment in fragments:
    if fragment['type'] != 'unmarked':
        tcolor = ob.colormap.get(fragment['type']) or 'unknown'
        ff = ob.color_match(fragment['pattern'], color=tcolor)
        outfragments.append(ff)
    else:
        outfragments.append(fragment['pattern'])
txt = " ".join(outfragments)  # may want to turn this in object property
print(txt)

<span style="color:rgb(178,223,138)"></span> <span style="color:rgb(178,223,138)"></span> <span style="color:rgb(178,223,138)"></span> <span style="color:rgb(178,223,138)"></span> <span style="color:rgb(166,206,227)"></span> <span style="color:rgb(166,206,227)"></span> <span style="color:rgb(166,206,227)"></span> 


In [41]:
%time runner.gather_found_delegates()


3. finding unmarked text
4. joining presidents and delegates
Phrase(, None)
Phrase(, None)
Phrase(, None)
Phrase(, None)
total 44 found 
CPU times: user 1min 1s, sys: 554 ms, total: 1min 1s
Wall time: 1min 3s


In [139]:
%time runner.verify_matches()

5. verifying matches
6. verifying spans
updating merged delegates database
7. finished to attendance lists and found database
CPU times: user 1min 22s, sys: 976 ms, total: 1min 23s
Wall time: 1min 25s


In [45]:
v = mg[3].variants['general'][0]
run_attendancelist.iterative_search('Noey', v)

1.0

In [157]:
mo.txt_to_fragments()

In [ ]:
js = runner.junksweeper
ms = runner.identify_delegates()
for T in runner.searchobs:
    searchob = runner.searchobs[T]
    mo = run_attendancelist.MatchAndSpan(so, junksweeper=js, previously_matched=runner.found_delegates,
                  match_search=ms)
    

In [ ]:
mo = run_attendancelist.MatchAndSpan(searchob.matched_text, junksweeper=js, previously_matched=runner.found_delegates,
                  match_search=ms)

In [143]:
mo = run_attendancelist.MatchAndSpan(so, junksweeper=js, previously_matched=runner.found_delegates,
                  match_search=ms)

In [146]:
so.get_unmatched_text()

['. ',
 ', De Heeren ',
 '. Vanden ',
 ', ',
 ' van Waveren. ',
 ', ',
 ', ',
 ', van HoornTaats van Amerongen, van Renswoude. ',
 ', ',
 '. ',
 ' , van lsselmuden, Ten ',
 '. ',
 ', ',
 ' van Steedum.']

In [155]:
for s in mo.nwsplit:
    if len(s) > 2:
        z = mo.previously_matched.variants.apply(lambda x: run_attendancelist.levenst_vals(x, s))
        r = mo.previously_matched.loc[z]
#         sr = mo.match_unmarked(s)
        print(s, ': ',r.max()['name'])

Heeren :  nan
Vanden :  nan
van Waveren :  nan
van HoornTaats :  nan
van Amerongen :  nan
van Renswoude :  van Reede tot Renswoude
van lsselmuden :  nan
Ten :  nan
van Steedum :  nan


In [154]:
for s in mo.nwsplit:
    if len(s) > 2:
        print(s)

Heeren
Vanden
van Waveren
van HoornTaats
van Amerongen
van Renswoude
van lsselmuden
Ten
van Steedum


In [118]:
mo.previously_matched.loc[mo.previously_matched.index.intersection(y)]

,ref_id,geboortejaar,sterfjaar,colleges,functions,period,active_stgen,was_gedeputeerde,variants,name,found_in,id,sg,period_active.left,period_active.right,hypothetical_life.left,hypothetical_life.right,p_interval.left,p_interval.right
0.0,13046,-102998.0,1751-10-06,"Admiraliteit op de Maze (1586-1795), Staten-Ge...","gecommitteerde, ordinaris gedeputeerde, lid","PeriodIndex(['1722-05-04', '1722-05-05', '1722...",True,True,"[2an Glinftra., an Glinjtra., Van Glinflra., Z...",van Glinstra,1728,13046,True,1722,1749,1688,1751,1722,1749
1.0,17862,-98615.0,1770-03-10,Staten-Generaal der Verenigde Nederlanden (158...,ordinaris gedeputeerde,"PeriodIndex(['1724-11-01', '1724-11-02', '1724...",True,True,"[Zan Wetderen., Van Welderen. P, Van Rechteren...",van Welderen,1728,17862,True,1724,1753,1700,1770,1724,1753


In [142]:
so = runner.searchobs['session-1730-01-05-num-1'].matched_text

In [64]:
ut = ''.join(unmarked)
st = re.sub(r"([A-Z]{1,20})", r" \1", ut)
splittekst = [x for x in re.split('\s|,|\.', st) if x != '']
print (splittekst)
# st = re.split(pattern='\s|\.', string=ut)
# it = ' '.join(st)
# splittekst = re.findall(pattern='[a-zA-Z][^A-Z]{2,}', string=it)
# print(ut, '\n', it, '\n', st, '\n', splittekst)
# for s in st:
#     if len(s) > 2 and len(js.find_matches(s, include_variants=True)) == 0:
#         mtch = runner.found_delegates.name.apply(lambda x: run_attendancelist.score_levenshtein_similarity_ratio(x,s) > 0.6)
#         tussenr = runner.found_delegates.loc[mtch]
#         print(s, tussenr)

['PRAESENTIRBRUS', 'De', 'Heeren', 'van', 'Lynden', 'van', "W'ynbergen", 'van', 'Helsdingen', 'PEES', 'De', 'Vander', 'Waayen', 'DE', 'Resofutien', 'gisteren', 'genomen']


In [176]:
lsr('met','van')

0.0

In [39]:
lsr = run_attendancelist.score_levenshtein_similarity_ratio
st = []
van = ''

st

[]

In [48]:
runner.verify_matches()

5. verifying matches
6. verifying spans
updating merged delegates database
7. finished to attendance lists and found database


In [ ]:
runner.presidents

In [ ]:
txt = "PRAESIDEDen Heere van Lynden PRAESENWIDBDU 8 De Heeren van Welderen van Singendonck Rasch Torck van Esk tot Norgena met twee extraordinana Gedepuieerden uyt de Provincie van Gelderlandt Van Weasscnaer le Leu de Wilbem vander Marck vander Ramborst Raadtpensionaris van Siingelandt Van Citters van Borsselen de Jongh Bout Noey Hungrosgne vun Hoorn Verelst met een entraordinaris Gedeputeerde uyt de Provincie wan Zeelandt Van Rensqwoude Sloterdyck met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt Bueisen Van Berchuys  Clant,"
pat = "%s(.*)%s"  # pat = "%s.*(.*)%s" 

In [ ]:
from republic.analyser.attendance_lists.pattern_finders import make_president_searcher
ps = make_president_searcher()
matches = ps.find_matches(text=txt, use_word_boundaries=False, include_variants=True)
presidents = [m for m in matches if m.label=='president']

president = best_match(presidents)
ofset = getattr(president, "offset")
prez = getattr(president, "string")
presentibi = [m for m in matches if m.label == 'presentibus']
if presentibi:
    presentibus = best_match(presentibi)
    prae = presentibus.string or ''
    end = getattr(presentibus, "offset") or len(txt)
spns = {}
if ofset != 0:
    preamble_span = (0, ofset - 1)
    preamble = txt[preamble_span[0]:preamble_span[1]]
    spns['preamble'] = (preamble, preamble_span)
pre_span = (ofset, ofset + len(prez))
spns['pre'] = (prez, pre_span)
presentibus_span = (end, end + len(prae))
spns['presentibus'] = (prae, presentibus_span)
searchpat = pat % (re.escape(prez), re.escape(prae))
r = re.search(searchpat, txt)
heer = r.group(1).strip()
heer

In [ ]:
prae

In [ ]:
mn = min([abs(len(m.phrase.exact_string) - len( m.string)) for m in presidents])

In [ ]:
candidates = [m for m in presidents if abs(len(m.phrase.exact_string) - len( m.string)) == mn]
candidate = max([m for m in candidates if score_match(m) == mx], key=lambda x: score_match(x))
candidate

In [14]:
runner.verify_matches()

5. verifying matches
verifying spans
updating merged delegates database
6. finished to attendance lists and found database


In [15]:
runner.all_matched

{12332: {'id': 12332,
  'm_kw': '',
  'score': 1.0,
  'name': 'Vegelin van Claerbergen',
  'variants': [('vander Steen', '', 0), ('vier', '', 0), ('wier', '', 0)]}}

In [ ]:
def besall_matchedatch(matches=[]):
    """
    calculates best match from fuzzysearch matches based on scorematch. Returns single match
    TODO: this does not account for best length if match scores are equal"""
    if matches:
        mx = max([score_match(m) for m in matches])
        candidates = [m for m in matches if score_match(m) == mx]
        candidate = max(candidates, key=lambda x: len(x.string))
        return candidate

In [ ]:
from republic.analyser.attendance

In [ ]:
from fuzzy_search.fuzzy_context_searcher import FuzzyContextSearcher
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_searcher import FuzzySearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

In [ ]:
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
fcs = FuzzyContextSearcher(config=config)
fps = FuzzyPhraseSearcher(config=config)

phrase = PhraseModel()

In [ ]:
from republic.data.delegate_database import ekwz
# create a list of domain keywords and phrases
# variants = [{'phrase': k, 'variants': v} for k, v in ekwz.items()]
# # create a PhraseModel object from the domain phrases
# phrase_model = PhraseModel(model=variants,)
#phrase_model.add_phrases(ekwz.keys())
# phrase_model.add_variants(variants)
fps.index_phrase_model(phrase_model=phrase_model)
fcs.index_phrase_model(phrase_model=phrase_model)

In [ ]:
vs = ['PRASIDE Den Heere',
          'PRESIDE Den Heere',
          'P R A E S I D E Den Heere',
          'PRAESIDE Den Heere',
          'P R A S I D E Den Heere',
          'P R E S I D E Den Heere',
          'P R AE S I D E Den Heere',
          'DR AS 1D E Den Heere',
          'PR ASL DE Den Heere',
          'PR A31 DE; Den Heere',
          'BR JE 3.1. DE, Den Heere']
vs = vs + ekwz['PRAS']
pvs = ['PRASENTIBUS',
                'PRESENTIBUS',
                'P R A E S E N T I B U S',
                'P RAE SE N TI B U S',
                'PRA&SENTIBUS']
variants = [{'phrase': 'PRAESIDE Den Heere', 'label':"president", 'variants': vs},
            {'phrase':'PRAESENTIBUS',  "variants":ekwz['PRASENTIBUS']}]
phrase_model = PhraseModel(model=variants)
phrase_model.json()

In [ ]:
p = run_attendancelist.ekwz['PRASENTIBUS']
pvs = ['PRASENTIBUS',
                'PRESENTIBUS',
                'P R A E S E N T I B U S',
                'P RAE SE N TI B U S',
                'PRA&SENTIBUS']
p+pvs

In [ ]:
from fuzzy_search.fuzzy_string import score_levenshtein_similarity_ratio

In [ ]:
res = fps.find_matches(text="PRAESIDE Den Heere, PRAESENTIBUS, PRESN", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
# for item in res:
#     print(item.score_levenshtein_similarity())
from scipy.special import softmax
from numpy import argmax
def score_match(match):
    result = sum([match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, score_levenshtein_similarity_ratio(str(match.variant.exact_string),str(match.phrase.exact_string))])
    return result

b = argmax(softmax([score_match(i) for i in res]))
res[b]

In [ ]:
score_levenshtein_similarity_ratio(str(i.variant.exact_string),str(i.phrase.exact_string))

In [ ]:
res = fps.find_matches(text="PRAESIDE Den Heere, PRAESENTIBUS, PRESN", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
mx = max([score_match(m) for m in res])
[m for m in res if score_match(m)==mx]
r = res[0]
r.end - r.offset
ranges = [set(range(r.offset, r.end)) for r in res]
p = run_attendancelist.pairwise
setpair = []
for pair in p(ranges):
    pair = list(pair)
    pair.sort()
    if pair[0].intersection(y):
        if pair[0].issubset(pair[1]):
            setpair.append(pair[1])
    

In [ ]:
matches=res
mx = max([score_match(m) for m in matches])
candidates = [m for m in matches if score_match(m)==mx]
candidate = max(candidates, key=lambda x: len(x.string))
candidate

In [ ]:
from itertools import combinations
set([x for x in combinations(ranges, 2) if set(x[0]) & set(x[1]) ])


In [ ]:
ranges
x = [{12, 13, 14, 15, 16},
 {12, 13, 14, 15, 16, 17}]
x.sort()
if x[0].issubset(x[1]):
    print(x[1],x)

In [ ]:
print(ranges, max(ranges, key=lambda x: len(x)))

In [ ]:
from heapq import merge


def find_overlap_and_non_overlap(ranges):
    starts, ends = zip(*ranges)
    overlap = range(max(starts), min(ends)+1)
    if not overlap:
        return ([], range(min(starts), max(ends)+1), [])

    less_non_overlap = range(*heapq.nsmallest(2, starts))
    end, start = heapq.nlargest(2, ends)
    greater_non_overlap = range(start+1, end+1)
    return (overlap, less_non_overlap, greater_non_overlap)


In [ ]:
ranges = [range(r.offset, r.end) for r in res]
find_overlap_and_non_overlap(ranges)

In [ ]:
item.score_levenshtein_similarity()

In [ ]:
# create a list of domain keywords and phrases
domain_phrases = [
    # terms for the chair and attendants of a meeting
    "PRAESIDE",
    "PRAESENTIBUS",
    # some weekdays in Latin
    "Veneris", 
    "Mercuri",
    # some date phrase where any date in January 1725 should match
    "den .. Januarii 1725"
]

# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(phrases=domain_phrases)

# register the phrase model with the searcher
fps.index_phrase_model(phrase_model)

# take some example texts: meetings of the Dutch States General in January 1725
text1 = "ie Veucris den 5. Januaris 1725. PR&ASIDE, Den Heere Bentinck. PRASENTIEBUS, De Heeren Jan Welderen , van Dam, Torck , met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Maasdam , vanden Boeizelaar , Raadtpenfionaris van Hoornbeeck , met een extraordinaris Gedeputeerde uyt de Provincie van Hollandt ende Welt-Vrieslandt. Velters, Ockere , Noey; van Hoorn , met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt. Van Renswoude , van Voor{t. Van Schwartzenbergh, vander Waayen, Vegilin Van I{elmuden. Van Iddekinge ‚ van Tamminga."

text2 = "Mercuri: den 10. Jangarii , | 1725. ia PRESIDE, Den Heere an Iddekinge. PRA&SENTIBUS, De Heeren /an Welderen , van Dam, van Wynbergen, Torck, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderland. Van Maasdam , Raadtpenfionaris van Hoorn=beeck. Velters, Ockerfe, Noey. Taats van Amerongen, van Renswoude. Vander Waasen , Vegilin, ’ Bentinck, van I(elmaden. Van Tamminga."

In [ ]:
t1 = runner.searchobs[T].matched_text.item
fps.find_matches(text=t1, include_variants=True, use_word_boundaries=False, allow_overlapping_matches=False)

In [ ]:
runner.junksweeper

In [ ]:
umt


In [ ]:
t1 = runner.searchobs[T].matched_text.item
fcs.find_matches(text=t1,include_variants=True, use_word_boundaries=False, allow_overlapping_matches=False)

In [ ]:
for phrase_dict in variants:
            main_phrase = as_phrase_object(phrase_dict, ngram_size=self.ngram_size, skip_size=self.skip_size)
            if main_phrase.phrase_string not in self.phrase_index:
                if add_new_phrases:
                    self.add_phrase(main_phrase)
                else:
                    continue
            if "variants" not in main_phrase.metadata:
                continue
            for variant_phrase_string in main_phrase.metadata["variants"]:
                variant_phrase = as_phrase_object(variant_phrase_string, ngram_size=self.ngram_size,
                                                  skip_size=self.skip_size)
                self.add_variant(variant_phrase, main_phrase)

In [ ]:
umt = runner.find_unmarked_text()
out = []
for s in umt:
    out.append(' '.join(s))
allout = ' '.join(out)
type(allout)

In [ ]:


# unmarked_text = ''.join(umt)
# splittekst = re.split(pattern="\s", string=unmarked_text)
# for s in splitteks
res = fuzzy_searcher.find_matches(text="we are all PRAESENTIBUS", include_variants=False, use_word_boundaries=False)
r = max(res, key=lambda x: x.score_levenshtein_similarity())
#fks.find_candidates(s, include_variants=True)

In [ ]:
res

In [ ]:
searchob = runner.searchobs['session-1730-02-14-num-1']
unmarked_text = ''.join(searchob.matched_text.get_unmatched_text())
print(unmarked_text)
splittekst = re.split(pattern="\s", string=unmarked_text)
for s in splittekst:
    if len(s) > 2 and len(fuzzy_searcher.find_matches(text=s, include_variants=False, use_word_boundaries=False)) == 0:
        print('ok')

In [ ]:
runner.junksweeper

In [ ]:
matchsearch = runner.identify_delegates()
for T in runner.searchobs:
    searchob = runner.searchobs[T]
    mo = run_attendancelist.MatchAndSpan(searchob.matched_text, junksweeper=runner.junksweeper, 
                                         previously_matched=runner.found_delegates, 
                                         match_search=matchsearch)

In [ ]:
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


In [ ]:
import statistics
from collections import Counter
txtlens = [len(runner.searchobs[T].text) for T in runner.searchobs]
print(min(txtlens), max(txtlens))
print(statistics.mean(txtlens))
print(statistics.stdev(txtlens))
print(Counter(txtlens).most_common())

In [ ]:
fuzzysearch_config = {
    "char_match_threshold": 0.7,
    "ngram_threshold": 0.5,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
from republic.analyser.attendance_lists.parse_delegates import make_junksweeper
from republic.data.delegate_database import ekwz
from republic.model.republic_phrase_model import month_names_early, month_names_late
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyKeywordSearcher
provincies = ['Holland', 'Zeeland', 'West-Vriesland', 'Gelderland', 'Overijssel', 'Utrecht', 'Friesland']
months = month_names_early + month_names_late
junksweeper = FuzzyKeywordSearcher(config=fuzzysearch_config)
indexkeywords = list(ekwz.keys()) + months + provincies
junksweeper.index_keywords(indexkeywords)
#ekwz = indexkeywords
fks = make_junksweeper(ekwz)


In [ ]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

# highger matching thresholds for higher quality OCR/HTR (higher precision, recall should be good anyway)
# lower matching thresholds for lower quality OCR/HTR (higher recall, as that's the main problem)
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# initialize a new searcher instance with the config
fuzzy_searcher = FuzzyPhraseSearcher(config)

# create a list of domain keywords and phrases
variants = [{'phrase': k, 'variants': v} for k, v in ekwz.items()]
# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(model=variants,)
#phrase_model.add_phrases(ekwz.keys())
# phrase_model.add_variants(variants)
fuzzy_searcher.index_phrase_model(phrase_model=phrase_model)

In [ ]:
res = fuzzy_searcher.find_matches(text="Heer. dikke Heeren", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
for item in res:
    print(item.score_levenshtein_similarity())

In [ ]:
item.score_levenshtein_similarity()

In [ ]:
# create a list of domain keywords and phrases
domain_phrases = [
    # terms for the chair and attendants of a meeting
    "PRAESIDE",
    "PRAESENTIBUS",
    # some weekdays in Latin
    "Veneris", 
    "Mercuri",
    # some date phrase where any date in January 1725 should match
    "den .. Januarii 1725"
]

# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(phrases=domain_phrases)

# register the phrase model with the searcher
fuzzy_searcher.index_phrase_model(phrase_model)

# take some example texts: meetings of the Dutch States General in January 1725
text1 = "ie Veucris den 5. Januaris 1725. PR&ASIDE, Den Heere Bentinck. PRASENTIEBUS, De Heeren Jan Welderen , van Dam, Torck , met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Maasdam , vanden Boeizelaar , Raadtpenfionaris van Hoornbeeck , met een extraordinaris Gedeputeerde uyt de Provincie van Hollandt ende Welt-Vrieslandt. Velters, Ockere , Noey; van Hoorn , met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt. Van Renswoude , van Voor{t. Van Schwartzenbergh, vander Waayen, Vegilin Van I{elmuden. Van Iddekinge ‚ van Tamminga."

text2 = "Mercuri: den 10. Jangarii , | 1725. ia PRESIDE, Den Heere an Iddekinge. PRA&SENTIBUS, De Heeren /an Welderen , van Dam, van Wynbergen, Torck, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderland. Van Maasdam , Raadtpenfionaris van Hoorn=beeck. Velters, Ockerfe, Noey. Taats van Amerongen, van Renswoude. Vander Waasen , Vegilin, ’ Bentinck, van I(elmaden. Van Tamminga."

In [ ]:
for phrase_dict in variants:
            main_phrase = as_phrase_object(phrase_dict, ngram_size=self.ngram_size, skip_size=self.skip_size)
            if main_phrase.phrase_string not in self.phrase_index:
                if add_new_phrases:
                    self.add_phrase(main_phrase)
                else:
                    continue
            if "variants" not in main_phrase.metadata:
                continue
            for variant_phrase_string in main_phrase.metadata["variants"]:
                variant_phrase = as_phrase_object(variant_phrase_string, ngram_size=self.ngram_size,
                                                  skip_size=self.skip_size)
                self.add_variant(variant_phrase, main_phrase)

In [ ]:
umt = runner.find_unmarked_text()
out = []
for s in umt:
    out.append(' '.join(s))
allout = ' '.join(out)
type(allout)

In [ ]:


# unmarked_text = ''.join(umt)
# splittekst = re.split(pattern="\s", string=unmarked_text)
# for s in splitteks
res = fuzzy_searcher.find_matches(text="we are all PRAESENTIBUS", include_variants=False, use_word_boundaries=False)
r = max(res, key=lambda x: x.score_levenshtein_similarity())
#fks.find_candidates(s, include_variants=True)

In [ ]:
res

In [ ]:
searchob = runner.searchobs['session-1730-02-14-num-1']
unmarked_text = ''.join(searchob.matched_text.get_unmatched_text())
print(unmarked_text)
splittekst = re.split(pattern="\s", string=unmarked_text)
for s in splittekst:
    if len(s) > 2 and len(fuzzy_searcher.find_matches(text=s, include_variants=False, use_word_boundaries=False)) == 0:
        print('ok')

In [ ]:
h = junksweeper.keyword_index['Holland']
h.

In [ ]:
splittekst = re.split(pattern="\s", string=unmarked_text)

In [ ]:
outdir = '/Users/rikhoekstra/republic_out'

In [ ]:
for yr in range(1729,1730):
    %time run_attendancelist.run(year=yr, outdir=outdir, verbose=False)

In [ ]:
import os
os.path.join('a', 'b')

In [ ]:
runner.moregentlemen

In [ ]:
score_levenshtein_distance_ratio('Amerongen','Borre van Amerongen')

In [ ]:
twijfelresults = []
for im in runner.all_matched:
    item = runner.all_matched[im]
    if score_levenshtein_distance_ratio(item['m_kw'], item['name'])<0.6:
        twijfelresults.append(im)
        

In [ ]:
len(runner.all_matched) - len(twijfelresults)

In [ ]:
runner.all_matched[17821]

In [ ]:
def match2pattern(match:dict, bestpattern='', year=0):
    goodresults = []
    quarantine = []
    variants = match['variants']
    if bestpattern == '':
            try:
                bestpattern = max(variants, key=lambda x: x[2])[0]
            except TypeError:
                print (variant)
                raise
    if match['score']>0.8:
        results = goodresults
    else:
        results = quarantine
    for variant in variants:
        if variant[2]>0.8:
            result = {}
            result['variant_pattern'] = variant[0]
            result['variant_score'] = variant[2]
            result['variant_keyword'] = bestpattern
            result['match_keyword']=match['m_kw']
            result['match_name'] = match['name']
            result['match_score'] = match['score']
            result['match_id'] = match['id']
            results.append(result)
    return goodresults, quarantine


In [ ]:
year = 1728
indexkws = []
qindexkws = []
for item in runner.all_matched:
    matches = runner.all_matched[item]
    i_kws, q_kws = match2pattern(matches, year=year)
    indexkws.extend(i_kws)
    qindexkws.extend(q_kws)

In [ ]:
from collections import defaultdict
goodvariants = {}
joined_good_variants = defaultdict(list)
unique_ids = list(set([i.get('match_id') for i in indexkws]))

for k in unique_ids:
    dd = defaultdict(list)
    ikws = [d for d in indexkws if d.get('match_id') == k]
    for d in ikws: # you can list as many input dicts as you want here
        for key, value in d.items():
            if key != 'match_id':
                dd[key].append(value)
    for kie, v in dd.items():
        v = list(set(v))
        if len(v) == 1:
            v = v[0]
        dd[kie] = v
    goodvariants[k] = dd

In [ ]:
goodvariants

In [ ]:
proposed_good_matches = []
for item in qindexkws:
    if item['variant_score'] > item['match_score']:
        item['match_keyword'] = item['variant_pattern']
        item['match_score'] = item['variant_score']
        proposed_good_matches.append(item)
        qindexkws.pop(qindexkws.index(item)) 
        

In [ ]:
vrs = [i['variant_pattern'] for i in indexkws]
vids = [i['match_id'] for i in indexkws]
prids = [i['match_id'] for i in proposed_good_matches]
vids.extend(prids)

In [ ]:
for item in qindexkws:
    if item['match_id'] in vids:
        print(item)

In [ ]:
len(qindexkws)

In [ ]:
len(proposed_good_matches)

In [ ]:
from statistics import mean
mean([i.get('match_score') for i in indexkws])

In [ ]:
min([i['match_score'] for i in indexkws])

In [ ]:
from republic.republic_keyword_matcher.finders.identify import iterative_search
reslt = {}
for i in twijfelresults:
    item = runner.all_matched[i]
    l = 0
    v = ''
    v = max(item['variants'], key=lambda z: len(z[0]))[0]
    reslt[v] = iterative_search(name=v, year=1728, debug=False, df=runner.abbreviated_delegates)

In [ ]:
from republic.republic_keyword_matcher.finders.identify import identify

In [ ]:
iterative_search(name='Taats', year=1728, df=runner.abbreviated_delegates)

In [ ]:
from IPython.core.display import display, HTML
for i in reslt:
    display(i, reslt[i])

G_heren = nx.Graph()
d_nodes = sorted(cl_heren)
for node in d_nodes:
    attached_nodes = cl_heren[node]
    G_heren.add_node(node)
    for nod in attached_nodes:
        G_heren.add_edge(node, nod)
pm_heren = list(nx.connected_components(G_heren))
pm_heren

In [ ]:
item

In [ ]:
from collections import Counter
lastfragments = []
lastpatterns = []
for ob in runner.searchobs:
    lastknownfragment = ''
    lastpattern = ''
    fragments = runner.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        f = item.get('delegate_name') or ''
        p = item.get('pattern') or ''
        if  f != '':
            lastknownfragment = item['delegate_name']
            lastfragments.append(lastknownfragment)
            lastpattern = p
            lastpatterns.append(lastpattern)
            break
Counter(lastfragments).most_common(10)

In [ ]:
for pers in Counter(lastfragments).most_common(10):
    print(pers, [goodvariants[d] for d in goodvariants if goodvariants[d]['match_name'].lower() == pers[0].lower()])
    #print(pers, [runner.all_matched[d] for d in runner.all_matched if runner.all_matched[d]['name'].lower() == pers[0].lower()])

In [ ]:
common_last_delegates = [pers[0] for pers in Counter(lastfragments).most_common(10)]

In [ ]:
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyKeywordSearcher
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
fks = FuzzyKeywordSearcher(config=fuzzysearch_config)
fks.use_word_boundaries = False
kws = [goodvariants[p]['match_keyword'] for p in goodvariants]
false_kws = ['PRAESIDE','PRAESENTIBUS', 'Nihil Actum']
kws.extend(false_kws)
fks.index_keywords(kws)
for kw in kws:
    if kw not in false_kws:
        z = [goodvariants[d] for d in goodvariants if goodvariants[d]['match_keyword'] == kw][0]
        vrs = z["variant_pattern"]
        for v in vrs:
            fks.index_spelling_variant(kw, v)

In [ ]:
suspect_obs = defaultdict(list)
for ob in runner.searchobs:
    fragments = runner.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        lastdelegate = ''
        ld = item.get('delegate_name')
        if ld in common_last_delegates:
            break
        elif ld == '': 
            p = item.get('pattern')
            r = fks.find_candidates(text=p)
            if len(r)>0:
                r = max(r, key=lambda x: x['levenshtein_distance'])
                if r["match_keyword"] in false_kws:
                    suspect_obs[r["match_keyword"]].append(ob)
                elif r["match_keyword"] in commonlastgoodvariantsvs:
                    begin = item['begin']
                    changeob = runner.searchobs[ob]
                    begin = res[-2]
                    for span in changeob.matched_text.spans:
                        if span.begin>=begin:
                            changeob.matched_text.spans.remove(span)

In [ ]:
suspect_obs

In [ ]:
for i in suspect_obs:
    print(runner.searchobs[i].get_fragments())

In [ ]:
begin = res[-2]
for span in changeob.matched_text.spans:
    if span.begin>=begin:
        changeob.matched_text.spans.remove(span)
changeob.matched_text.spans

In [ ]:
res[2]

In [ ]:
commonlastgoodvariants = {v:goodvariants[v] for v in goodvariants if goodvariants[v]['match_name'] in common_last_delegates}

In [ ]:
commonlastgoodvariantsvs = []
for v in goodvariants:
    if goodvariants[v]['match_name'] in common_last_delegates:
        commonlastgoodvariantsvs.extend(goodvariants[v]['variant_pattern'])
commonlastgoodvariantsvs

In [ ]:
lastspans = [f[-1].to_json() for f in out]

In [ ]:
lastfragments[-100]a

In [ ]:
spans = []
for T in ra.searchobs:
    ob = ra.searchobs[T]
    spans.append(ob.get_spans())

In [ ]:
noprez = {}
for T in ra.searchobs:
    ob = ra.searchobs[T]
    p = [s for s in ob.get_spans() if s.type=='president']
    if p:
        noprez[T] = p
    else:
        noprez[T] = None


In [ ]:
upr = list(set(ra.presidents))

In [ ]:
ks = list(noprez.keys())
ks.sort()


In [ ]:
noprez[t][0]

In [ ]:
fks.keyword_index.keys()

In [ ]:
# this does not generalize much, but we'll leave that for later
from collections import defaultdict
from republic.republic_keyword_matcher.finders.pattern_finders import make_herensearcher
from republic.fuzzy.fuzzy_keyword_searcher import score_levenshtein_distance_ratio


class DelegateChecker(object):
    def __init__(self, keywords, searchobs):
        self.searcher = make_herensearcher(keywords=keywords)
        self.context_searcher = make_herensearcher(keywords={'PRAESENTIBUS':[], 'Den Heere':[]})
        self.searchobs = searchobs
        self.likely_prez = defaultdict(list)
        self.checkdeps = self.make_noprez()
        self.ks = self.make_keys()
        
    def make_keys(self):
        ks = list(self.checkdeps.keys())
        ks.sort() # we depend on the key order
        return ks
        
    def get_vicinity_presidents(self,t):
        mn = max(self.ks.index(t)-7, 0) 
        mx = min(self.ks.index(t)+7, len(ks)-1)
        likely_prez = defaultdict(list)
        for pt in ks[mn:mx]:
            if self.checkdeps[pt]:
                president = self.checkdeps[pt][0]
                name = president.delegate_name
                pattern = president.pattern
                likely_prez[name].append(pattern)
        return likely_prez
        # for p in likely_prez:
        #     ra.all_matched
#         xsisting_kws = list(self.searcher.keyword_index.keys())
#         kws = [k for k in list(likely_prez.keys()) if k not in xsisting_kws]            
#         self.searcher.index_keywords(kws)
#         for k in kws:
#             for variant in likely_prez[k]:
#                 self.searcher.index_spelling_variant(k, variant=variant)
    
    def check(self):
        results = []
        for t in self.ks:
            if not self.checkdeps[t]:
                txt = self.searchobs[t].text
                if len(txt) > 10:
                    res = s.find_candidates(text=txt)
                    if len(res) == 0:
                        break
                    hres = [r for r in res if r['match_keyword']=='Den Heere']
                    sres = min(hres, key=lambda x: x['match_offset'])
                    if len(sres) > 0:
                        start = sres['match_offset']  + len(sres['match_string'])
                    else:
                        start = 0
                    pres = [r for r in res if r['match_keyword']=='PRAESENTIBUS']
                    if len(pres) > 0:
                        eres = min(pres, key=lambda x: x['match_offset'])
                        end = eres['match_offset']
                    else:
                        end = start + 20
                    stxt = txt[start:end]
                    stxt = stxt.strip()
                likely_prez = self.get_vicinity_presidents(t)
                res = {}
                for cand in likely_prez:
                    candidate = likely_prez[cand]
                    for i in candidate:
                        res[i] = score_levenshtein_distance_ratio(i, stxt)
                chosen = max(res, key=lambda x: res[x])
                score = res[chosen]
                if score > 0.6:
#                     for cand in likely_prez:
#                         candidate = max([(i, score_levenshtein_distance_ratio(stxt, i)) for i in likely_prez[cand]])
                    candidate = chosen
                else:
                    prop = self.searcher.find_candidates(stxt)
                    if len(prop)>0:
                        candidate = max(prop, key=lambda x: x['levenshtein_distance'])
                    else:
                        candidate = None
                results.append({'txt': stxt,
#                                'lp':likely_prez,
                                'search result': candidate}) 
        return results

    
    def make_noprez(self):
        noprez = {}
        for T in self.searchobs:
            ob = self.searchobs[T]
            p = [s for s in ob.get_spans() if s.type=='president']
            if p:
                noprez[T] = p
            else:
                noprez[T] = None
        return noprez

In [ ]:
s = make_herensearcher(keywords={'PRAESENTIBUS':[], 'Den Heere':[]})
res = [r for r in s.find_candidates(keyword="Den Heere", text="Den Heere Jan Steenbergen. PRAZSENTIBUS, De Heeren Van Lynden, van Singeudonck, van Heeckeren tot de") if r['match_keyword']=='Den Heere']
sres = min(res, key=lambda x: x['match_offset'])
sres['match_offset']  + len(sres['match_string'])

In [ ]:
keywords = {i.name:[v.form for v in i.variants['general']] for i in ra.moregentlemen}


In [ ]:
keywrds = {}
for e in ra.all_matched:
    i = ra.all_matched[e]
    nm = i["name"]
    f = [i["m_kw"]]
    f.extend([v[1] for v in i["variants"]])
    keywrds[nm]=list(set(f))
    

In [ ]:
keywrds['Tamminga van Alberda']

In [ ]:
dlgchk = DelegateChecker(keywords=keywrds, searchobs=ra.searchobs)

In [ ]:
dlgchk.searcher.find_candidates("Van Schwartzenbergh. ")

In [ ]:
d = {'Velters': ['Velters', 'Velters', 'Velters', 'Velters'], 'van Wassenaer van Obdam': ['Van Wassenaer', 'Van Wassenaer'], 'van Reede tot Renswoude': ['Van Renswoude', 'Van Renswoude', 'Van Renswonde']}

In [ ]:
txt = ' Van Renswoude.'
res = {}
for cand in d:
    candidate = d[cand]
    for i in candidate:
        res[i] = score_levenshtein_distance_ratio(i, txt)

In [ ]:
max(res, key=lambda x: res[x])

In [ ]:
dlgchk.check()

In [ ]:
len("'Van Lynden, van Singendonck, IS ter Vergaderinge gelesen de Requeste van Johan Coenraadts Hendricks")

In [ ]:
score_levenshtein_distance_ratio('Van Schwartzenbergh', 'Van Schwart zenbergh')

In [ ]:

fks = make_herensearcher(keywords={i.name:[v.form for v in i.variants['general']] for i in ra.moregentlemen})

In [ ]:
make_herensearcher(keywords={})

In [ ]:
for t in ks:
    if not noprez[t]:
        likely_prez = get_vicinity_presidents(noprez, t, )
        txt = ra.searchobs[t].text
        if len(txt) > 10:
            e = txt.find('PRAESENTIBUS')
            if e > -1:
                stxt = txt[:e]
            else: 
                stxt = txt[:100]
            print(likely_prez.find_candidates(stxt), stxt)
                
         

In [ ]:
likely_prez

In [ ]:
for k in ra.searchobs:
    so = ra.searchobs[k]
    sspans = so.get_spans()
    for s in sspans:
        if s.pattern in s.delegate_name:
            if s.delegate_name > s.pattern
            so.get_txt

In [ ]:
ra.all_matched

vars2graph(cl_heren) 

existing_herensearcher = FuzzyKeywordSearcher(config=fuzzysearch_config)
existing_herensearcher.index_keywords(pm_heren)

In [ ]:
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

## retrieve presentielijsten and convert to objects

In [ ]:
searchobs = make_presentielijsten(year='1728')

In [ ]:
searchobs['session-1728-01-01-num-1']

### 1. Find presidents

In [ ]:
presidents = president_searcher(presentielijsten=searchobs)

In [ ]:
heren = presidents


In [ ]:
heren = [h.strip() for h in heren]

### 2.find provincial extraordinaris gedeputeerden

In [ ]:
ps = province_searcher(presentielijsten=searchobs)

### 3. make a list of remaining words from the attendents lists

This collects all non-'identified words' from the searchobjects

In [ ]:
unmarked = make_groslijst(presentielijsten=searchobs)

In [ ]:
# filter out words that occur less than 5 times 
from collections import Counter
c = Counter(unmarked)
len([k for k in c if c[k]>3])

In [ ]:
filtered_text = " ".join(list(c.keys()))

In [ ]:
fks = FuzzyKeywordSearcher(fuzzysearch_config)
tussenkeys = fks.find_close_distance_keywords(list(c.keys()))


In [ ]:
dralist = vars2graph(tussenkeys)


In [ ]:
with open('republic_junk.json','r') as rj:
    ekwz = json.load(fp=rj)
provincies=['Holland','Zeeland','West-Vriesland','Gelderland','Overijssel', 'Utrecht','Friesland']
from republic.model.republic_phrase_model import week_day_names, month_names_early, month_names_late, spelling_variants
months = month_names_early + month_names_late 
junksweeper = FuzzyKeywordSearcher(config=fuzzysearch_config)
junksweeper.index_keywords(list(ekwz.keys()))
for key in ekwz.keys():
    for variant in ekwz[key]:
        junksweeper.index_spelling_variant(keyword=key, variant=variant)
junksweeper.index_keywords(months)
junksweeper.index_keywords(provincies)

In [ ]:
def get_lscore(r):
    return r['levenshtein_distance']

rawres = []
for t in dralist:
#    t = rawlist[i]
    if len(t)>1:
        rawtext = ' '.join(t)
    else: 
        rawtext = t[0]
    r = junksweeper.find_candidates(rawtext)
    try:
        nr = max(r, key=get_lscore)
        if nr['levenshtein_distance'] < 0.5:
            rawres.append(t)
    except ValueError:
        rawres.append(t)

In [ ]:
#deputies = fks.find_close_distance_keywords(rawlist)
deputies = rawres

We now have:

1. __heren__: a list of presidents
1. __ezkw__: a rough list of words that have been identified as junk before
1. __deputies__: a rough list of words that are presumably delegates

### 5. cluster them into connected variants. Ideally these would contain the different variants of deputies

In [ ]:
cl_heren = fks.find_close_distance_keywords(heren)
G_heren = nx.Graph()
d_nodes = sorted(cl_heren)
for node in d_nodes:
    attached_nodes = cl_heren[node]
    G_heren.add_node(node)
    for nod in attached_nodes:
        G_heren.add_edge(node, nod)
#list(nx.connected_components(G_heren))

#which clusters them nicely

### 6. Identify the attendance lists with this overview of deputies 

In [ ]:
abbreviated_delegates = pd.read_pickle('sheets/abbreviated_delegates.pickle')

In [ ]:
#previously_matched = pd.read_excel('sheets/xml_herkend.xlsx')
previously_matched = pd.read_pickle('sheets/found_deputies.pickle')
previously_matched['id'] = previously_matched['ref_id']
previously_matched.columns

pm_heren = list(previously_matched['name'].unique())

In [ ]:
TUSSENVOEGSELS = [
    'à',
    '\xe0',
    r'd\'',
    r'de',
    r'den',
    r'der',
    r'des',
    r'di',
    r'en', 
    r'het',
    r"in 't",
    r'la',
    r'la', 
    r'le',
    r'of',
    r'van',
    r'ten', 
    r'thoe', 
    r'tot', 
    r"'t",
]


In [ ]:
stopwords = TUSSENVOEGSELS
def nm_to_delen(naam, stopwords=stopwords):
    nms = [n for n in naam.split(' ') if n not in stopwords]
    #nms.append(naam)
    return nms
           
keywords = list(abbreviated_delegates.name)
kwrds = {key:nm_to_delen(key) for key in keywords}


In [ ]:
nwkw = {d:k for k in list(set(keywords)) for d in k.split(' ') if d not in stopwords}

In [ ]:
# note that this config is more lenient than the default values 
fuzzysearch_config = {'char_match_threshold': 0.7,
 'ngram_threshold': 0.5,
 'levenshtein_threshold': 0.5,
 'ignorecase': False,
 'ngram_size': 2,
 'skip_size': 2}

In [ ]:
herensearcher = FuzzyKeywordSearcher(config=fuzzysearch_config)
exclude = ['Heeren', 'van', 'met', 'Holland']
filtered_kws = [kw for kw in nwkw.keys() if kw not in exclude]
herensearcher.index_keywords(keywords=filtered_kws)
for k in filtered_kws:
    for variant in nwkw[k]:
        herensearcher.index_spelling_variant(k,variant=variant)

In [ ]:
from collections import defaultdict
transposed_graph = defaultdict(list)
for node, neighbours in kwrds.items():
    for neighbour in neighbours:
        transposed_graph[neighbour].append(node)

In [ ]:
day = 1728
year = day

In [ ]:

dayinterval = pd.Interval(day, day, closed="both")
register = {}
pats = []

In [ ]:
# setup presidents for matching
existing_herensearcher = FuzzyKeywordSearcher(config=fuzzysearch_config)
existing_herensearcher.index_keywords(pm_heren)
connected_presidents = list(nx.connected_components(G_heren))
matchfnd = FndMatch(year=year,
                 #patterns=pats,
                 register=register,
                 rev_graph=transposed_graph,
                 searcher=herensearcher,
                 junksearcher=junksweeper,
                 df=abbreviated_delegates)

In [ ]:
found_presidents = find_delegates(input=connected_presidents,
                                  matchfnd=matchfnd,
                                  df=abbreviated_delegates,
                                  previously_matched=previously_matched,
                                  year=1728)

### Cluster deputies

We now do the same trick with the delegates. 

In [ ]:
found_delegates = find_delegates(input=deputies,
                                 matchfnd=matchfnd,
                                 df=abbreviated_delegates,
                                 previously_matched=previously_matched,
                                 year=1728 )

### join presidents and deputies


In [ ]:
all_matched = {}
for d in [found_presidents['matched'], found_delegates['matched']]:
    for key in d:
        if key not in all_matched.keys():
            all_matched[key]= d[key]
        else:
            all_matched[key]['variants'].extend(d[key]['variants'])

In [ ]:
moregentlemen = [MatchHeer(all_matched[d]) for d in all_matched.keys() if type(d)==int] # strange keys sneak in

In [ ]:
mh_df = pd.DataFrame(all_matched).transpose().reset_index()

In [ ]:
matchsearch = FuzzyKeywordSearcher(config=fuzzysearch_config)
kws = defaultdict(list)
matcher = {}
for key in all_matched:
    variants = all_matched[key].get('variants')
    keyword = all_matched[key].get('m_kw')
    idnr = all_matched[key].get('id')
    name = all_matched[key].get('name')
    for variant in variants:
        kws[keyword].append(variant[0])
        matcher[variant[0]] = {'kw':keyword, 'id':idnr, 'name':name}
matchsearch.index_keywords(list(kws.keys()))
for k in kws:
    for v in kws[k]:
        matchsearch.index_spelling_variant(keyword=k,variant=v)

In [ ]:
deputyregister = defaultdict(list)
search_results = {}
# fm = FndMatch(year=1728,
#                  #patterns=pats,
#                  register=register,
#                  rev_graph=transposed_graph,
#                  searcher=herensearcher,
#                  junksearcher=junksweeper,
#                  df=abbreviated_delegates)
for T in searchobs:
    ob = searchobs[T].matched_text
    mo = MatchAndSpan(ob, junksweeper=junksweeper, previously_matched=previously_matched, match_search=matchsearch)
#     for span in ob.spans:
#         if span.type in ['president', 'delegate']:
#             if span.pattern == '':
#                 span.set_pattern(pattern=ob.item[span.begin:span.end])
#             r = fm.match_candidates(heer=span.pattern)
#             span.set_delegate(delegate_id=r.id, 
#                               delegate_name=r.proposed_delegate, 
#                               delegate_score=r.score)

### 8. Mark delegates 

In [ ]:
delresults = Counter()
for T in searchobs.keys():
    searchob = searchobs[T]
    result = get_delegates_from_spans(searchob.matched_text)
    try:
        id  = result['delegate']['id']
        if id:
            delresults.update([id])
    except KeyError:
        pass
delresults

previously_matched.loc[previously_matched.id.isin(delresults.keys())]
abbreviated_delegates.loc[abbreviated_delegates.id.isin(delresults.keys())]

In [ ]:
framed_gtlm = pd.DataFrame(moregentlemen)

In [ ]:
framed_gtlm.rename(columns={0:'gentleobject'}, inplace=True)

In [ ]:
framed_gtlm['vs'] = framed_gtlm.gentleobject.apply(lambda x: [e for e in x.variants['general']])
framed_gtlm['ref_id'] = framed_gtlm.gentleobject.apply(lambda x: x.heerid)
#framed_gtlm['uuid'] = framed_gtlm.gentleobject.apply(lambda x: x.get_uuid())
framed_gtlm['name'] = framed_gtlm.gentleobject.apply(lambda x: x.name)
framed_gtlm['found_in'] = day
framed_gtlm

In [ ]:
def levenst_vals(x, txt):
    scores= [score_levenshtein_distance_ratio(i.form,txt) for i in x if len(i.form)>1]
    if max(scores) > 0.8:
        return max(scores)
    else:
        return np.nan


In [ ]:
def delegates2spans(searchob):
    spans = searchob.matched_text.spans
    for span in spans:
        txt = searchob.matched_text.item[span.begin:span.end]
        msk = framed_gtlm.vs.apply(lambda x: levenst_vals(x, txt))
        mres = framed_gtlm.loc[msk==msk.max()]
        if len(mres)>0:
    #     if len(mres)>0:
    #         setattr(span, 'delegate_id', mres.uuid.iat[0])
    #         setattr(span, 'delegate_name', mres.name.iat[0])
        #print(kand, all_matched[kand])
            span.set_pattern(txt)
        if len(mres)>0:
            span.set_delegate(delegate_id=mres.ref_id.iat[0], delegate_name=mres.name.iat[0])


In [ ]:
for T in searchobs:
    searchob = searchobs[T]
    #mo = MatchAndSpan(ob, junksweeper=junksweeper, previously_matched=previously_matched)
    delegates2spans(searchob)

In [ ]:
merged_deps = pd.merge(left=framed_gtlm, right=abbreviated_delegates, left_on="ref_id", right_on="id", how="left")

In [ ]:
serializable_df = merged_deps[['ref_id','geboortejaar', 'sterfjaar', 'colleges', 'functions',
       'period', 'sg', 'was_gedeputeerde', 'p_interval', 'h_life', 'vs','name_x', 'found_in',]]
serializable_df.rename(columns={'name_x':'name', 
                                'vs':'variants', 
                                "h_life": "hypothetical_life", 
                                "p_interval":"period_active", 
                                "sg": "active_stgen"},  inplace=True)


In [ ]:
serializable_df.to_pickle('sheets/found_deputies.pickle')


In [ ]:
for interval in ["hypothetical_life", "period_active"]:
    serializable_df[interval]=serializable_df[interval].apply(lambda x: [x.left, x.right])
serializable_df.to_excel('sheets/found_deputies.xlsx')

In [ ]:
out=[]
for T in searchobs:
    ob = searchobs[T]
    out.append(ob.to_dict())

In [ ]:
#with open('/Users/rikhoekstra/Downloads/1728_pres2.json', 'w') as fp:
json_year = json.dumps(obj=out)

In [ ]:
with open('/Users/rikhoekstra/Downloads/1728_pres_20210420.json', 'w') as fp:
    json.dump(fp=fp, obj=out)

In [ ]:
delegate_json = serializable_df[['ref_id', 'name', 'geboortejaar', 'sterfjaar', 'colleges',
       'functions', 'active_stgen', 'was_gedeputeerde',
       'period_active', 'hypothetical_life']].to_json(orient="records")

In [ ]:
searchobs = runner.searchobs

In [ ]:
searchobs = runner.searchobs
from IPython.display import HTML
t_out=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"""\n<tr><td><strong>{T}</strong></td><td>{rest}</td>"""
    if url:
        rest += f"""<td><a href='{url}'>link naar {T}-image</a></td>"""
    rest += "</tr>"
    t_out.append(rest)
#out.reverse()
outtable = "".join(t_out)
HTML(f"<table>{outtable}</table>")


In [ ]:
with open(f"/Users/rikhoekstra/Downloads/{day}_check.html", 'w') as flout:
    flout.write(f"<html><body><h1>results for {day}</h1>\n")
    flout.write(f"<table>{outtable}</table>")
    flout.write("</body></html>")

## To Elasticsearch

In [ ]:
from republic.republic_keyword_matcher.elastic_search_helpers import bulk_upload
local_republic_es = Elasticsearch()
local_republic_es

In [ ]:
local_republic_es.indices.create(index='attendancelist')


In [ ]:
data_dict = out
bulk_upload(bulkdata=data_dict, index='attendancelist', doctype='attendancelist')